In [122]:
import numpy as np

#System size
L = 4 #Hilbert space dimension is 2**L

#Constructing z-basis states
bin_set=[]
for i in range(2**L, 2**(L+1)):
    bin_set.append(bin(i)[3:])

basis=[]
for binary in bin_set:
    basis.append([int(digits) for digits in binary][::-1])

#Test initial state
test = np.arange(2**L)

In [129]:
#Setting the permutation action of spin operators
def pre_sigma(bin_seq,subspace):
    output = np.empty(len(bin_seq))
    output[:] = bin_seq[:] 
    if output[subspace] == 0:
        output[subspace] = 1
    else:
        output[subspace] = 0 
    return output.astype(int)

pre_flipped_assign = np.array([np.array([pre_sigma(binary,sbspce) for binary in basis]) for sbspce in range(L)])
dec_converter = np.array([2**n for n in range(L)])
multiplied = dec_converter*pre_flipped_assign

#Cheat sheet for the sigma function
permutations = np.array([[np.sum(bin_arrays) for bin_arrays in multiplied[sbspce]] for sbspce in range(L)]).astype(int)
tracker = 2*(np.array(basis).transpose()-1/2) #Records the initial state of flipped state. "0" states are mapped to -1, "1 states are mapped to +1".

#Defining the spin operators
def sigma(vector,*xyz_subspace):
    global L, permutations, tracker
    
    input_vec = np.empty(2**L).astype(complex)
    input_vec[:] = vector[:]
    output = np.empty(2**L).astype(complex)
    
    
    for entry in xyz_subspace[::-1]:
        xyz = entry[0]
        subspace = entry[1]
        
        if subspace == 4: #Imposing periodic boundary condition
            subspace = 0
        
        for i in range(2**L):
            if xyz == 1: #spin x
                output[permutations[subspace,i]] = input_vec[i]

            if xyz == 2: #spin y
                output[permutations[subspace,i]] = (1j*input_vec*tracker[subspace])[i]

            if xyz == 3: #spin z
                output[i] = (input_vec*tracker[subspace])[i]
        
        input_vec[:] = output[:]

    return output

#Constructing a function that adds sigma maps
def sigma_add(vector,*xyz_subspace_set):
    output = np.zeros(2**L).astype(complex)
    
    for xyz_subspace in xyz_subspace_set:
        output[:] += sigma(vector,*xyz_subspace)
        
    return output

In [124]:
#Defining alpha, beta, and gamma functions
def alpha(theta,phi):
    return 1
def beta(theta,phi):
    return 1
def gamma(theta,phi):
    return 1

In [125]:
#Constructing operators coupled with the alpha, beta, and gamma functions
alpha_map_entries = [[[2,i]] for i in range(L)]
beta_map_entries = [[[1,i],[2,i+1]] for i in range(L)]+[[[2,i],[1,i+1]] for i in range(L)]
gamma_map_entries = [[[3,i],[2,i+1]] for i in range(L)]+[[[2,i],[3,i+1]] for i in range(L)]

def alpha_map(vector):
    return sigma_add(vector,*alpha_map_entries)
def beta_map(vector):
    return sigma_add(vector,*beta_map_entries)
def gamma_map(vector):
    return sigma_add(vector,*gamma_map_entries)

In [131]:
alpha_map(test)

array([0.+15.j, 0.+17.j, 0.+19.j, 0.+15.j, 0.+23.j, 0.+15.j, 0.+15.j,
       0. +1.j, 0.+31.j, 0.+15.j, 0.+15.j, 0. -7.j, 0.+15.j, 0.-11.j,
       0.-13.j, 0.-45.j])